In [16]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.naive_bayes import GaussianNB

In [17]:
data=pd.read_csv("C:\\Users\\Farida\\Documents\\Colleage\\Semester 4\\Pattern Recognition\\adult\\adult.data", header=None)
data.columns=["age", "type_employer", "fnlwgt", "education",
                "education_num","marital", "occupation", "relationship", "race","sex",
                "capital_gain", "capital_loss", "hr_per_week","country", "income"]
print(data.head)

<bound method NDFrame.head of        age      type_employer  fnlwgt    education  education_num  \
0       39          State-gov   77516    Bachelors             13   
1       50   Self-emp-not-inc   83311    Bachelors             13   
2       38            Private  215646      HS-grad              9   
3       53            Private  234721         11th              7   
4       28            Private  338409    Bachelors             13   
...    ...                ...     ...          ...            ...   
32556   27            Private  257302   Assoc-acdm             12   
32557   40            Private  154374      HS-grad              9   
32558   58            Private  151910      HS-grad              9   
32559   22            Private  201490      HS-grad              9   
32560   52       Self-emp-inc  287927      HS-grad              9   

                   marital          occupation    relationship    race  \
0            Never-married        Adm-clerical   Not-in-family   Wh

In [18]:
test=pd.read_csv("C:\\Users\\Farida\\Documents\\Colleage\\Semester 4\\Pattern Recognition\\adult\\adult.test", header=None)
test.columns=["age", "type_employer", "fnlwgt", "education",
                "education_num","marital", "occupation", "relationship", "race","sex",
                "capital_gain", "capital_loss", "hr_per_week","country", "income"]
print(test.head)

<bound method NDFrame.head of        age  type_employer  fnlwgt      education  education_num  \
0       25        Private  226802           11th              7   
1       38        Private   89814        HS-grad              9   
2       28      Local-gov  336951     Assoc-acdm             12   
3       44        Private  160323   Some-college             10   
4       18              ?  103497   Some-college             10   
...    ...            ...     ...            ...            ...   
16276   39        Private  215419      Bachelors             13   
16277   64              ?  321403        HS-grad              9   
16278   38        Private  374983      Bachelors             13   
16279   44        Private   83891      Bachelors             13   
16280   35   Self-emp-inc  182148      Bachelors             13   

                   marital          occupation     relationship  \
0            Never-married   Machine-op-inspct        Own-child   
1       Married-civ-spouse     

In [19]:
x_train=data.drop('income',axis='columns')
x_test=test.drop('income',axis='columns')
y_train = data['income'].apply(lambda x: 1 if x.strip() == '>50K' else 0)
y_test = test['income'].apply(lambda x: 1 if x.strip() == '>50K.' else 0)

In [20]:
combined_values=pd.concat([x_train,x_test],axis=0,ignore_index=True)
print(combined_values.head)

<bound method NDFrame.head of        age      type_employer  fnlwgt   education  education_num  \
0       39          State-gov   77516   Bachelors             13   
1       50   Self-emp-not-inc   83311   Bachelors             13   
2       38            Private  215646     HS-grad              9   
3       53            Private  234721        11th              7   
4       28            Private  338409   Bachelors             13   
...    ...                ...     ...         ...            ...   
48837   39            Private  215419   Bachelors             13   
48838   64                  ?  321403     HS-grad              9   
48839   38            Private  374983   Bachelors             13   
48840   44            Private   83891   Bachelors             13   
48841   35       Self-emp-inc  182148   Bachelors             13   

                   marital          occupation     relationship  \
0            Never-married        Adm-clerical    Not-in-family   
1       Married-civ

In [21]:

categorical_columns=combined_values.select_dtypes(exclude=[np.number]).columns
numerical_columns=combined_values.select_dtypes(include=[np.number]).columns
encoder=OneHotEncoder(drop='first', sparse=False)
encoded_data=encoder.fit_transform(combined_values[categorical_columns])
encoded_df=pd.DataFrame(encoded_data,columns=encoder.get_feature_names_out(categorical_columns))
data_encoded=pd.concat([combined_values[numerical_columns],encoded_df],axis=1)
print(data_encoded.head)

<bound method NDFrame.head of        age  fnlwgt  education_num  capital_gain  capital_loss  hr_per_week  \
0       39   77516             13          2174             0           40   
1       50   83311             13             0             0           13   
2       38  215646              9             0             0           40   
3       53  234721              7             0             0           40   
4       28  338409             13             0             0           40   
...    ...     ...            ...           ...           ...          ...   
48837   39  215419             13             0             0           36   
48838   64  321403              9             0             0           40   
48839   38  374983             13             0             0           50   
48840   44   83891             13          5455             0           40   
48841   35  182148             13             0             0           60   

       type_employer_ Federal-gov

c:\Users\Farida\Documents\Anaconda\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [22]:
x_train_encoded = data_encoded.iloc[:len(x_train), :].reset_index(drop=True)
x_test_encoded = data_encoded.iloc[len(x_train):, :].reset_index(drop=True)

In [23]:
model = GaussianNB()
model.fit(x_train_encoded, y_train)
accuracy = model.score(x_test_encoded, y_test)
print("Accuracy:", accuracy)
predictions = model.predict(x_test_encoded)
print("Predictions:", predictions)

Accuracy: 0.7957127940544193
Predictions: [0 0 0 ... 0 1 0]


In [24]:
tn,fp,fn,tp=confusion_matrix(y_test,predictions).ravel()
sensitivity=tp/(tp+fn)
print("Sensitivity = ",sensitivity)
specificity=tn/(tn+fp)
print("Specificity = ",specificity)
posterior_probability=model.predict_proba(x_test_encoded)
print("posterior probability = ",posterior_probability)

Sensitivity =  0.3049921996879875
Specificity =  0.9474869320466426
posterior probability =  [[9.97412227e-01 2.58777336e-03]
 [9.84756080e-01 1.52439199e-02]
 [9.93541986e-01 6.45801379e-03]
 ...
 [9.78079367e-01 2.19206332e-02]
 [1.21026493e-05 9.99987897e-01]
 [9.71722872e-01 2.82771280e-02]]
